参考代码：https://aistudio.baidu.com/aistudio/projectdetail/1986156?channel=0&channelType=0&shared=1


**赛题简述**

	PALM病理性近视病灶检测与分割常规赛的重点是研究和发展与病理性近视诊断和患者眼底照片病变分割相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上检测病理性近视的自动算法。具体任务为：检测眼底图像是否出现视网膜萎缩病变和脱离病变，若有，需要实现病变区域的分割。
    
![](https://ai-studio-static-online.cdn.bcebos.com/bcb4c79dd34242e0bd83db8ea8062d7fa6a12a59999c44f09a73c9298a45fae3)
    
    
**数据基本标签**

	萎缩病变分割金标准：萎缩区域：0；背景：255；
	脱离病变分割金标准：脱离区域：0；背景：255。
    
**训练数据集**
文件名称：Train

Train里有两个文件夹，一个是fundus_images，一个是Lesion_Masks。

* fundus_images文件夹内包含**800**张眼底彩照，分辨率为1444×1444，或2124×2056。命名形如H0001.jpg、N0001.jpg、P0001.jpg和V0001.jpg。

* Lesion_Masks文件夹内包含两个文件夹：**Atrophy**和**Detachment**，其中，Atrophy文件夹包含fundus_images里眼底彩照的萎缩病变区域分割金标准，大小与对应的眼底彩照一致。命名前缀与对应眼底图像一致，后缀为bmp。同理，Detachment文件夹包含fundus_images里眼底彩照的脱离病变区域分割金标准，大小与对应的眼底彩照一致，命名前缀与对应眼底图像一致，后缀为bmp。**请注意，若Lesion_Masks中无某张眼底图像的病灶分割结果，说明该图像中不包含对应的病灶区域**。

**测试数据集**

文件名称：PALM-Testing400-Images.zip

压缩包里包含400张眼底彩照，命名形如T0001.jpg。




# 基线内容方案

* 解压数据与数据划分

    -- # 解压数据集

    -- !unzip -oq /home/aistudio/data/data85135/常规赛：PALM病理性近视病灶检测与分割.zip -d PaddleSeg/data

    -- # 划分数据

    -- !python utils/dataset_splited.py

* 数据标签预处理

    -- # 转换标签

    -- !python utils/dataset_pretrans.py

    * 原分类为1分类问题，为了问题研究的充分性和更大程度上利用多分类间的类别竞争对分类结构有一个更好的指导

    * 二分类问题描述，原标签为0不变，将255无效值转换为1值

    * 后期提交前会后处理，消去1值，换回赛题需要的255值

* 利用PaddleSeg套件加速赛题开发与测试: 使用套件config中的模型yml进行快速高效的实验开发——注意数据集yml的配置

* 实现训练流程

* 实现预测流程

* 完成提交结果 -- 基线方案为0.67+的得分  调整后 得分0.69574

    -- # 提交结果后处理

    -- utils/post_process.py
    

In [ ]:
# 删除data目录 —— 用于数据划分时，产生了意料之外的数据扩充时的数据重置
%cd /home/aistudio/
!rm -rf PaddleSeg/data
# 解压数据集到PaddleSeg目录下的data文件夹
!unzip -oq /home/aistudio/data/data93538/常规赛：PALM病理性近视病灶检测与分割.zip -d PaddleSeg/data

/home/aistudio


# 划分数据

划分比例为0.7 ——可在utils文件夹下的dataset_splited.py修改**train_percent**为其它值

数据预处置-可在utils文件夹下的dataset_pretrans.py中查看相关代码--实现将255转化为1，原问题变二分类问题

> 注意：当前数据处理中，存在对数据进行扩充，因此当前程序运行一次之后会使得原分类数据数目增加——(扩增不宜过大，否则划分数据进行训练和验证时会出现偏差问题)

> 仅可运行一次，多次运行会导致填充数量过多，这是源码中扩充机制决定的(直接扩充到源文件夹中，所以下一次划分的时候就会默认把源文件中所有的文件读取)

> 感兴趣可前往查看dataset_splited.py的数据扩充区

In [ ]:
# 保证路径为初始路径
%cd /home/aistudio

# 划分数据
!python utils/dataset_splited.py

# 转换标签--预处置
!python utils/dataset_pretrans.py

/home/aistudio
————开始数据清洗划分————
BMP标注数据: Detachment: 18张 	 Atrophy: 582张
———— 开始构建数据目录 ————
The Dir Has Create: PaddleSeg/data/Detachment/Image 
The Dir Has Create: PaddleSeg/data/Detachment/Label 
The Dir Has Create: PaddleSeg/data/Detachment/Test 
The Dir Has Create: PaddleSeg/data/Atrophy/Image 
The Dir Has Create: PaddleSeg/data/Atrophy/Label 
The Dir Has Create: PaddleSeg/data/Atrophy/Test 
————开始针对类别Detachment进行数据扩充(3张)————
Processing Train-Detachment Split: 100%|██████| 800/800 [00:18<00:00, 44.38it/s]
实际扩充: 213 张, 现类别 Detachment 拥有数据: 21 份
————开始针对类别Atrophy进行数据扩充(32张)————
Processing Train-Atrophy Split: 100%|█████████| 800/800 [00:18<00:00, 42.18it/s]
实际扩充: 210 张, 现类别 Atrophy 拥有数据: 614 份
——开始划分Detachment——
The Split Params: train_percent=0.70
Processing Test-Detachment Split: 100%|██████| 400/400 [00:00<00:00, 598.04it/s]
——开始划分Atrophy——
The Split Params: train_percent=0.70
Processing Test-Atrophy Split: 100%|█████████| 400/400 [00:00<00:00, 616.86it/s]
PaddleSeg/data
├── Atrop

移除原数据，减小项目空间，减少下一次进入和退出保存时花的时间

In [ ]:
# 移除’常规赛：PALM病理性近视病灶检测与分割‘文件夹
!rm -rf PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割
!rm -rf PaddleSeg/data/__MACOSX 


## 启动训练

In [ ]:
# 下载依赖项，保证PaddleSeg正常运行
%cd PaddleSeg
%pwd
!pip install -r requirements.txt

/home/aistudio/PaddleSeg
Looking in indexes: https://mirror.baidu.com/pypi/simple/


> 更换自定义的模型文件时，只需要修改对应的模型yml、数据集yml(_base_中的yml)、以及替换下方的yml即可进行训练了

> 不要更改输出目录，否则后边的代码也需要修改，包括预测、后处理中的所有路径

In [ ]:
%cd /home/aistudio/PaddleSeg
!python train.py --c /home/aistudio/dnlnet_res50_D.yml \
--use_vdl \
--save_interval 200 \
--do_eval \
--save_dir ./output/Detachment

In [ ]:
# 训练分割Atrophy的模型，并保存到/output/Atrophy
#已经存放了训练后最优模型 路径在PaddleSeg/output
!python train.py --c /home/aistudio/dnlent_res50_A.yml \
--use_vdl \
--save_interval 400 \
--do_eval \
--save_dir ./output/Atrophy

##  开始预测

这可以直接使用emanet进行预测，不用修改数据集yml，也不用修改相应的模型yml，注意训练权重对应即可！

> 预测结果按类别分别放在`./output/result/Detachment` 和 `./output/result/Atrophy` 下

> 如果使用不同的模型对不同的分割类进行讨论，注意模型yml即可

> 提交结果为两种单独预测的结果

**默认使用Iou评估最好的模型训练参数--best_model**

In [ ]:
%cd /home/aistudio/PaddleSeg/
# 预测Detachment
!python predict.py --config /home/aistudio/dnlnet_res50_D.yml \
--model_path output/Detachment/best_model/model.pdparams \
--image_path data/Detachment/Test \
--save_dir ./output/result/Detachment

/home/aistudio/PaddleSeg
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-07-30 01:09:16 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 2000
learning_rate:
  decay:
    power: 0.9
    type: poly
  value: 0.01
loss:
  coef:
  - 1
  - 0.4
  types:
  - type: CrossEntropyLoss
  - type: CrossEntropyLoss
model:
  backbone:
    output_stride: 8
    pretrained: https://bj.b

In [ ]:
# 预测Atrophy
!python predict.py --config /home/aistudio/dnlent_res50_A.yml  \
--model_path output/Atrophy/best_model/model.pdparams \
--image_path data/Atrophy/Test \
--save_dir ./output/result/Atrophy

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-07-30 01:12:09 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 15000
learning_rate:
  decay:
    power: 0.9
    type: poly
  value: 0.01
loss:
  coef:
  - 1
  - 0.4
  types:
  - type: CrossEntropyLoss
  - type: CrossEntropyLoss
model:
  backbone:
    output_stride: 8
    pretrained: https://bj.bcebos.com/paddleseg/dygr

# 五、预测结果后处理

* 将类别值1换为255，进行赛题结果提交

In [ ]:
%cd /home/aistudio/
!python utils/post_process.py

/home/aistudio
————开始提交结果前的后处理————
————开始Detachment预测结果后处理————
100%|█████████████████████████████████████████| 400/400 [00:24<00:00, 16.48it/s]
————开始Atrophy预测结果后处理————
100%|█████████████████████████████████████████| 400/400 [00:24<00:00, 16.63it/s]
后处理完成(cost: 48.740721464157104 s)！


#  提交比赛结果

In [ ]:
# 复制文件到最顶层目录
%cd /home/aistudio
!cp -r PaddleSeg/output/result/ Lesion_Segmentation

# 过程移动文件--保证不包含生成的子目录
%cd Lesion_Segmentation
!cp -r Detachment/pseudo_color_prediction/. Detachment
!cp -r Atrophy/pseudo_color_prediction/. Atrophy

# 获取指定的提交目录格式
!rm -rf Detachment/added_prediction
!rm -rf Detachment/pseudo_color_prediction

!rm -rf Atrophy/added_prediction
!rm -rf Atrophy/pseudo_color_prediction

# 压缩文件
%cd /home/aistudio
!zip -r Lesion_Segmentation.zip Lesion_Segmentation
# 删除复制的文件
!rm -rf Lesion_Segmentation

/home/aistudio
/home/aistudio/Lesion_Segmentation
/home/aistudio
updating: Lesion_Segmentation/ (stored 0%)
updating: Lesion_Segmentation/Detachment/ (stored 0%)
updating: Lesion_Segmentation/Detachment/T0234.png (deflated 55%)
updating: Lesion_Segmentation/Detachment/T0305.png (deflated 67%)
updating: Lesion_Segmentation/Detachment/T0337.png (deflated 98%)
updating: Lesion_Segmentation/Detachment/T0207.png (deflated 80%)
updating: Lesion_Segmentation/Detachment/T0079.png (deflated 98%)
updating: Lesion_Segmentation/Detachment/T0085.png (deflated 65%)
updating: Lesion_Segmentation/Detachment/T0081.png (deflated 61%)
updating: Lesion_Segmentation/Detachment/T0393.png (deflated 63%)
updating: Lesion_Segmentation/Detachment/T0338.png (deflated 98%)
updating: Lesion_Segmentation/Detachment/T0349.png (deflated 75%)
updating: Lesion_Segmentation/Detachment/T0193.png (deflated 98%)
updating: Lesion_Segmentation/Detachment/T0260.png (deflated 78%)
updating: Lesion_Segmentation/Detachment/T0013